Show how to create a collection and upload data to it

In [1]:
import sys, pathlib

repo_root = pathlib.Path().resolve()
repo_root = repo_root.parent

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from indexing.qdrant import load_qdrant_client, create_qdrant_collection, delete_collection
from indexing.upload import upload_points, create_chunk_id
from preprocessing.chunking import chunking_text
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse
from embeddings.embedding import FastEmbedEmbeddings, output_supported_models

/home/hlm/Documents/Mini-RAG/.rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = load_qdrant_client()
text = chunking_text(chunk_size=500, chunk_overlap=60)

Output possible model

In [ ]:
output_supported_models("dense") #or dense

In [ ]:
output_supported_models("sparse") #or dense

In [4]:
model_dense = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")
model_sparse = FastEmbedSparse(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")

In [5]:
create_qdrant_collection(client, collection_name="RAG", model_dense=model_dense, model_sparse=model_sparse)

Collection config updated with new model


For filtering purpose, I need to create a payload

In [ ]:
#Right now, I will only filter on the type but next I will use advanced filtering
#Need to create payload before adding points because it will add the id directly for better filtering
client.create_payload_index(collection_name="RAG", field_name="metadata.type", field_schema=models.PayloadSchemaType.KEYWORD, wait=True)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
vector_store = QdrantVectorStore(client=client, collection_name="RAG", retrieval_mode=RetrievalMode.HYBRID, embedding=model_dense, sparse_embedding=model_sparse)

In [8]:
upload_points(vector_store, text)

Uploading batches: 100%|██████████| 30/30 [26:39<00:00, 53.31s/it]
